# Notebook to access to data 

- Confluence page : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/372867091/System-level+Science+Performance+Verification+Sprint
- slides : https://docs.google.com/presentation/d/1bPjS5NGtiEL2lfjmvP1UYdG_tMEDfZxX26ayhf7MhtY/edit#slide=id.ga2f7fb1a1f_0_70

- where to find the campains
- https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

- plot Navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator/plot/%2Frepo%2Fmain/LSSTComCam%2Fruns%2FDRP%2FDP1%2Fw_2025_05%2FDM-48666/objectTableCore_coaddInputCount_SkyPlot

- Notebooks examples
- https://github.com/lsst-dm/DMTR-401/blob/main/notebooks/test_LVV-T40_T1240.ipynb
- https://github.com/lsst-dm/DMTR-412/blob/tickets/DM-38728/notebooks/test_LVV-T1751_AM1_AM2.ipynb

- author : Sylvie Dagoret-Campagne
- creattion date : 2025-02-03
- last update : 2025-04-20

In [ ]:
# Confirm that the version of the Science Pipelines is recent:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom

In [ ]:
import os
import gc
import glob
import numpy as np
import pandas as pd
import random

import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
from astropy.visualization import (MinMaxInterval, AsinhStretch, ZScaleInterval, LogStretch, LinearStretch,
                                   ImageNormalize)

In [ ]:
# Set plotting defaults
%matplotlib inline
import matplotlib.pyplot as plt
zscale = ZScaleInterval()

# Set up some plotting defaults:
plt.rcParams.update({'figure.figsize' : (12, 8)})
plt.rcParams.update({'font.size' : 24})
plt.rcParams.update({'axes.linewidth' : 3})
plt.rcParams.update({'axes.labelweight' : 3})
plt.rcParams.update({'axes.titleweight' : 5})
plt.rcParams.update({'ytick.major.width' : 3})
plt.rcParams.update({'ytick.minor.width' : 2})
plt.rcParams.update({'ytick.major.size' : 8})
plt.rcParams.update({'ytick.minor.size' : 5})
plt.rcParams.update({'xtick.major.size' : 8})
plt.rcParams.update({'xtick.minor.size' : 5})
plt.rcParams.update({'xtick.major.width' : 3})
plt.rcParams.update({'xtick.minor.width' : 2})
plt.rcParams.update({'xtick.direction' : 'in'})
plt.rcParams.update({'ytick.direction' : 'in'})


In [ ]:
# check where to look in : https://usdf-rsp.slac.stanford.edu/plot-navigator

#repo = '/repo/main'
#collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666'

#repo = '/repo/dp1'
#collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260'

repo = '/repo/main'
collection = "LSSTComCam/runs/DRP/DP1/w_2025_15/DM-50050"


In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
for _ in registry.queryCollections():
    if collection  in _:
        print(_)

In [ ]:
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collection).any(
        execute=False, exact=False
    ):
        # Limit search results to the data products
        if (
            ("_config" not in datasetType.name)
            and ("_log" not in datasetType.name)
            and ("_metadata" not in datasetType.name)
            and ("_resource_usage" not in datasetType.name)
        ):
            print(datasetType)

In [ ]:
did_tract2394 = {'instrument':'LSSTComCam', 'tract':2394, 'skymap':'lsst_cells_v1'}
metric_extract2394 = butler.get('matchedVisitCore_metrics', collections=collection, dataId=did_tract2394)

In [ ]:
metric_extract2394 

In [ ]:
print('Tract 2394:\n')
for met in metric_extract2394['stellarAstrometricRepeatability1']:
    if ('AM1' in met.metric_name.metric) and \
       (met.metric_name.metric.startswith('g_') or met.metric_name.metric.startswith('r_') or \
        met.metric_name.metric.startswith('i_')):
        print(met.datum)
print('\n')
for met in metric_extract2394['stellarAstrometricRepeatability2']:
    if ('AM2' in met.metric_name.metric) and \
       (met.metric_name.metric.startswith('g_') or met.metric_name.metric.startswith('r_') or \
        met.metric_name.metric.startswith('i_')):
        print(met.datum)

